In [3]:
#from sklearn.experimental import enable_iterative_imputer
#from sklearn.preprocessing import FunctionTransformer
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer #, IterativeImputer
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, OneHotEncoder, StandardScaler, MaxAbsScaler

from sklearn.metrics import precision_score, recall_score, accuracy_score, classification_report, ConfusionMatrixDisplay
from sklearn.metrics import roc_auc_score, roc_curve, auc, SCORERS
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, ExtraTreesClassifier

from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn import set_config
set_config(display ="diagram")

import matplotlib.pyplot as plt
%matplotlib inline

In [4]:

full_df=pd.read_csv('../data/processed/crashes.gz', compression='gzip', low_memory=False)

In [108]:
sample_df=full_df.sample(8000, random_state=100)
#sample_df=df.copy()

y = sample_df['GUILTY']
x = sample_df.drop(['GUILTY'],axis=1)

X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=100, test_size=0.25, stratify=y)


In [109]:
numeric_columns = ['NUM_PASSENGERS','SUN_GLARE']

nominal_columns = ['DEVICE_CONDITION', 'FIRST_CRASH_TYPE', 'FIRST_CONTACT_POINT', 
           'TRAFFICWAY_TYPE','ROADWAY_SURFACE_COND','VEHICLE_DEFECT',
             'DRIVERS_LICENSE_CLASS', 'AGE_GROUP', 'VEHICLE_AGE', 'AIRBAG_DEPLOYED',
            'PHYSICAL_CONDITION', 'MANEUVER',  'DRIVER_VISION',  'ALIGNMENT' ,  'TRAFFIC_CONTROL_DEVICE']


X_train=X_train[numeric_columns+nominal_columns]
X_test=X_test[numeric_columns+nominal_columns]

X_train=X_train.apply(LabelEncoder().fit_transform)
X_test=X_test.apply(LabelEncoder().fit_transform)

In [110]:
from sklearn.base import BaseEstimator, TransformerMixin

class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names=attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values
    
    

num_pipeline = Pipeline([
        ('selector', DataFrameSelector(numeric_columns)),
        ('imputer', SimpleImputer(strategy = 'constant', fill_value=0))
 #       ('std_scaler', StandardScaler())
    ])

cat_pipeline = Pipeline([
        ('selector', DataFrameSelector(nominal_columns)),
#        ('imputer', SimpleImputer(strategy = 'constant', fill_value='NA')),
#        ('cat_encoder', OneHotEncoder(sparse=False, handle_unknown =  'ignore' )),
    ])


fu = FeatureUnion(transformer_list=[
        ("num_pipeline", num_pipeline),
        ("cat_pipeline", cat_pipeline),
    ])




In [111]:
#X_train_transformed = fu.fit_transform(X_train[numeric_columns+nominal_columns])
#X_train.info(verbose=True, show_counts=True)

#columns=fu.transformer_list[1][1][2].get_feature_names_out()
#columns=np.insert(columns, 0, 'Intercept')
#transformed_df=pd.DataFrame(X_train_transformed, columns=X_train[numeric_columns+nominal_columns].columns)
transformed_df=pd.DataFrame(X_train_transformed)

#transformed_df.info()
#transformed_df.info(verbose=True, show_counts=True)


In [118]:
from sklearn.naive_bayes import GaussianNB
bayesian_pipe = Pipeline([
                          ('fu', fu),
#                           ('label_encoder', ModifiedLabelEncoder),
                          ('model', BaggingClassifier(QuadraticDiscriminantAnalysis(), 
                                                      n_estimators=100, ))
 
                              ])


In [119]:
pipe_grid = {}

#pipe_grid = {'model__criterion': ['gini', 'entropy']}

gs_pipe = GridSearchCV(estimator=bayesian_pipe,
                       param_grid=pipe_grid, cv=5)


#pd.DataFrame(SCORERS.values())

In [120]:
gs_pipe.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('fu',
                                        FeatureUnion(transformer_list=[('num_pipeline',
                                                                        Pipeline(steps=[('selector',
                                                                                         DataFrameSelector(attribute_names=['NUM_PASSENGERS',
                                                                                                                            'SUN_GLARE'])),
                                                                                        ('imputer',
                                                                                         SimpleImputer(fill_value=0,
                                                                                                       strategy='constant'))])),
                                                                       ('cat_pipeline',
                                                                        Pipeline(steps=[('selector',
                                                                                         DataFrameSelector(attribute_names=['DEVICE_CONDITIO...
                                                                                                                            'FIRST_CONTACT_POINT',
                                                                                                                            'TRAFFICWAY_TYPE',
                                                                                                                            'ROADWAY_SURFACE_COND',
                                                                                                                            'VEHICLE_DEFECT',
                                                                                                                            'DRIVERS_LICENSE_CLASS',
                                                                                                                            'AGE_GROUP',
                                                                                                                            'VEHICLE_AGE',
                                                                                                                            'AIRBAG_DEPLOYED',
                                                                                                                            'PHYSICAL_CONDITION',
                                                                                                                            'MANEUVER',
                                                                                                                            'DRIVER_VISION',
                                                                                                                            'ALIGNMENT',
                                                                                                                            'TRAFFIC_CONTROL_DEVICE']))]))])),
                                       ('model',
                                        BaggingClassifier(base_estimator=QuadraticDiscriminantAnalysis(),
                                                          n_estimators=100))]),
             param_grid={})

In [121]:
y_pred_train=gs_pipe.predict(X_train)
y_pred_test=gs_pipe.predict(X_test)

In [122]:
gs_pipe.best_params_


{}

In [123]:
print(gs_pipe.score(X_train, y_train ))
print(classification_report(y_train, y_pred_train))
print('------------')
print(classification_report(y_test, y_pred_test))

0.7036666666666667
              precision    recall  f1-score   support

           0       0.72      0.90      0.80      3951
           1       0.63      0.32      0.42      2049

    accuracy                           0.70      6000
   macro avg       0.67      0.61      0.61      6000
weighted avg       0.69      0.70      0.67      6000

------------
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1317
           1       0.34      1.00      0.51       683

    accuracy                           0.34      2000
   macro avg       0.17      0.50      0.25      2000
weighted avg       0.12      0.34      0.17      2000



C:\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
